<a href="https://colab.research.google.com/github/PTC-Education/PTC-API-Playground/blob/main/Onshape_Countdown_DX_in_Education_Summit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Onshape Countdown Timer
Controlling [this public Onshape document](https://cad.onshape.com/documents/7b1ea499ba204910f3db0450/w/fbe1acea8e7fe33ef70a7741/e/434578822c9ea104cadd843a) using snippets from the Onshape API Snippets library found [in this repo](https://github.com/PTC-Education/PTC-API-Playground).


import client

In [ ]:
!pip install onshape-client
from onshape_client.client import Client
import json
base = 'https://cad.onshape.com' # change this if you're using a document in an enterprise (i.e. "https://ptc.onshape.com")

add application keys, stored in "colabkeys.py" file, uploaded to runtime in "files" on the left bar.

In [3]:
import os
def include(filename):
    if os.path.exists(filename): 
        execfile(filename)
include('colabkeys.py')
client = Client(configuration={"base_url": base,
                               "access_key": access,
                               "secret_key": secret})

/usr/local/lib/python3.7/dist-packages/onshape_client/client.py:158: UserWarning: A Client was already created so this will create another and override it. Please use Client.get_client() to get the previously created client.
  "A Client was already created so this will create another and override it. Please use "


Create function for getting the information for the sketch with text in the first feature of a Part Studio, changing the text to the input timeRemaining, then sending the updated feature to the Part Studio. Also tracking how long the function takes to execute and return elapsed time.

In [27]:
def updateTime(timeRemaining):
  start = time.time()
  fixed_url = '/api/partstudios/d/did/w/wid/e/eid/features'

  #https://cad.onshape.com/documents/7b1ea499ba204910f3db0450/w/fbe1acea8e7fe33ef70a7741/e/434578822c9ea104cadd843a
  did = '7b1ea499ba204910f3db0450'
  wid = 'fbe1acea8e7fe33ef70a7741'
  eid = '434578822c9ea104cadd843a'

  fixed_url = fixed_url.replace('did', did)
  fixed_url = fixed_url.replace('wid', wid)
  fixed_url = fixed_url.replace('eid', eid)

  method = 'GET'
  params = {}
  payload = {}
  headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
            'Content-Type': 'application/json'}

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)
  # print(json.loads(response.data)["features"][0]["message"]["entities"][0]["message"]['parameters'][1]['message']['value'])

  newFeature = json.loads(response.data)["features"][0]
  newFeature["message"]["entities"][0]["message"]['parameters'][1]['message']['value'] = '''Next Session Starts in
'''+timeRemaining
  # print(newFeature["message"]["entities"][0]["message"]['parameters'][1]['message']['value'])
  # print(timeRemaining)
  # print(json.loads(response.data)["sourceMicroversion"])
  # print(json.loads(response.data)["serializationVersion"])

  fixed_url = '/api/partstudios/d/did/w/wid/e/eid/features/featureid/fid'

  fid = json.loads(response.data)["features"][0]["message"]["featureId"]

  fixed_url = fixed_url.replace('did', did)
  fixed_url = fixed_url.replace('wid', wid)
  fixed_url = fixed_url.replace('eid', eid, 1)
  fixed_url = fixed_url.replace('fid', fid)

  # print(fixed_url)

  method = 'POST'
  params = {}
  payload = {'feature': newFeature,
            'serializationVersion': json.loads(response.data)["serializationVersion"],
            'sourceMicroversion': json.loads(response.data)["sourceMicroversion"]}
  headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
            'Content-Type': 'application/json'}

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)
  # print(response.data)
  end = time.time()
  callTime = start-end
  return callTime

Function for counting down. Calls updateTime funciton above and accounts for the API call time in the timer.

In [ ]:
# import the time module
import time

# define the countdown func.
def countdown(timerIn):
  t = int(timerIn)
  while t > 0:
    mins, secs = divmod(t, 60)
    timer = '{:02d}:{:02d}'.format(mins, secs)
    print(timer)
    callTime = -updateTime(timer)
    print(callTime)
    time.sleep(4)
    timerIn -= 4+callTime
    t = int(timerIn)
    
  print('Right now!')
  updateTime('Right now!')


# input time in seconds
timerIn = input("Enter the time in seconds: ")

# function call
countdown(float(timerIn))
